In [41]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [42]:
df= pd.read_csv('merged_v0.46.csv', index_col='ID')
# gfm= pd.read_csv('gfm_v0.1.csv')

/home/allen/Documents/Programs/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (1,2,8,13,14,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
df= df[df.COUNTRY=='United States of America']

In [44]:
len(df)

156416

In [5]:
df.index= np.arange(len(df))

In [50]:
df.to_csv('USFD_v1.0.csv')

# Include FEDB

In [8]:
base_dir= 'FEDB'

In [9]:
_dir= '0101/0101.shp'

In [46]:
for repo in os.listdir(base_dir):
    fdir= os.path.join(base_dir,repo,repo+'.shp')
    events= gpd.read_file(fdir)
    _slice= slice(len(df), len(df)+len(events),1)
    df= df.reindex(range(0, len(df)+len(events)))
#     df.loc[len(df):len(df)+len(events),:]= ''
#     df.loc[_slice, 'SOURCE_DB']= 'FEDB'
    df.loc[_slice, 'SOURCE_ID']= events.STCD.to_list()
    df.loc[_slice, 'SOURCE_DB']= 'FEDB'
    df.loc[_slice, 'DATE_BEGIN']= pd.to_datetime(events.StartTimeF).dt.strftime('%Y%m%d%H%M%S').to_list()
    df.loc[_slice, 'DATE_END']= pd.to_datetime(events.EndTimeF).dt.strftime('%Y%m%d%H%M%S').to_list()
    df.loc[_slice, 'DURATION']= (pd.to_datetime(events.EndTimeF)-pd.to_datetime(events.StartTimeF)).dt.days.to_list()
    df.loc[_slice, 'LON']= [np.array(events.geometry[i].coords)[0][0] for i in range(len(events))]
    df.loc[_slice, 'LAT']= [np.array(events.geometry[i].coords)[0][1] for i in range(len(events))]

In [47]:
len(df)

698508

# Correct duration for GFM

# State-wise figures

In [89]:
df= pd.read_csv('USFD_v1.0.csv', index_col='ID')

/home/allen/Documents/Programs/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (1,2,6,7,8,13,14,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
df['COUNTRY']= 'United States of America'

In [86]:
def fillStates(state):
    masks= gdf[pd.isna(gdf.STATE)].geometry.unique().within(state.geometry)
    print('done %s'%state[0])
    
    return masks, state[0]

In [87]:
results= states.apply(fillStates, axis=1)

done Connecticut
done Massachusetts
done Maine
done New Hampshire
done Vermont
done New Jersey
done Pennsylvania
done New York
done Indiana
done Illinois
done Ohio
done Michigan
done Wisconsin
done Iowa
done Missouri
done Kansas
done Nebraska
done Minnesota
done North Dakota
done South Dakota
done North Carolina
done South Carolina
done Georgia
done Maryland
done Virginia
done Florida
done District of Columbia
done Delaware
done West Virginia
done Tennessee
done Kentucky
done Alabama
done Mississippi
done Arkansas
done Texas
done Oklahoma
done Louisiana
done New Mexico
done Colorado
done Nevada
done Utah
done Arizona
done Montana
done Wyoming
done Idaho
done Washington
done Oregon
done California


In [146]:
unique_geometry= gdf[pd.isna(gdf.STATE)].geometry.unique()

In [145]:
df.loc[[156424, 156443, 156445, 156447, 156450]]

,DATE_BEGIN,DATE_END,DURATION,LON,LAT,COUNTRY,STATE,LOCATION,AREA,FATALITY,...,SOURCE_DB,SOURCE_ID,DESCRIPTION,slope,dem,LULC,DISTANT_RIVER,CONT_AREA,DEPTH,year
ID,,,,,,,,,,,,,,,,,,,,,
156424,2.00312e+13,20040112053000,14.0,-84.820833,41.945833,United States of America,Nebraska,NaN,NaN,1.0,...,FEDB,4096515.0,NaN,0.415662,305.0,40.0,2.104688,3.296512,NaN,1999.0
156443,2.00712e+13,20080522143000,152.0,-84.820833,41.945833,United States of America,Nevada,NaN,NaN,2.0,...,FEDB,4096515.0,NaN,0.415662,305.0,40.0,2.104688,3.296512,NaN,1999.0
156445,2.00809e+13,20080919103000,6.0,-84.820833,41.945833,United States of America,Minnesota,NaN,NaN,0.0,...,FEDB,4096515.0,NaN,0.415662,305.0,40.0,2.104688,3.296512,NaN,1999.0
156447,2.00902e+13,20090328113000,29.0,-84.820833,41.945833,United States of America,Iowa,NaN,NaN,1.0,...,FEDB,4096515.0,NaN,0.415662,305.0,40.0,2.104688,3.296512,7.91,1999.0
156450,2.00906e+13,20090628023000,8.0,-84.820833,41.945833,United States of America,Alabama,NaN,NaN,4.0,...,FEDB,4096515.0,NaN,0.415662,305.0,40.0,2.104688,3.296512,NaN,1999.0


In [ ]:
for (masks, state_name) in results:
    for geometry in unique_geometry[masks]:
        if len(geometry.coords)>=1:
            ids= gdf[(gdf.geometry==geometry)].index.values
#             print(ids)
            df.loc[ids,'STATE']= state_name

In [132]:
df.STATE

ID
0               Oklahoma
1                Indiana
2               Kentucky
3         North Carolina
4               NEW YORK
               ...      
698503               NaN
698504               NaN
698505               NaN
698506               NaN
698507               NaN
Name: STATE, Length: 698508, dtype: object

In [60]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.LON, df.LAT))

In [58]:
def countByStates(state, country):
    masks= gdf[gdf.COUNTRY==country].geometry.within(state.geometry)
    _gdf= gdf[gdf.COUNTRY==country][masks]
    
    return len(_gdf)

def getIndByStates(state, country):
    masks= gdf[gdf.COUNTRY==country].geometry.within(state.geometry)
    
    return gdf[gdf.COUNTRY==country][masks].index.values

In [56]:
states= gpd.read_file('US_floods_states')

In [61]:
states['events']= states.apply(countByStates, axis=1)

TypeError: countByStates() missing 1 required positional argument: 'country'